In [1]:
# 使用的框架paddle
# 相关框架的加载
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import os
import gzip
import json
import random
import numpy as np

In [3]:
# 数据加载
def load_data(mode='train', batch_size = 100):
    """ 
    加载数据函数
    传入两个参数mode,batch_size
    """
    datafile = './mnist .json.gz'
    print(f'加载数据从{datafile}')
    # 加载json数据文件
    data = json.load(gzip.open(datafile))
    print(f'加载数据集完毕')

    # 根据输入的参数进行数据的划分
    train_set, val_set, eval_set = data
    if mode == 'train':
        imgs, labels = train_set[0], train_set[1]
    elif mode == 'valid':
        imgs, labels = val_set[0], val_set[1]
    elif mode == 'eval':
        imgs, labels = eval_set[0], eval_set[1]
    else:
        raise Exception("mode 参数必须为['train','valid', 'eval']")
    print(f"加载的数据集数量{len(imgs)}")

    # 获取数据集的长度
    imgs_length = len(imgs)

    # 给每一条数据编号
    index_list = list(range(imgs_length))
    BATCHSIZE = batch_size

    # 定义数据生成器
    def data_generator():
        if mode == 'train':
            # 训练模式下要打乱数据
            random.shuffle(index_list)
        imgs_list = []
        labels_list = []
        for i in index_list:
            # 处理数据
            img = np.array(imgs[i]).astype('float32')
            label = np.array(labels[i]).astype('float32')
            imgs_list.append(img)
            labels_list.append(label)
            if len(imgs_list) == BATCHSIZE:
                # 获得一个BATCHSIZE的数据
                yield np.array(imgs_list),np.array(labels_list)
                # 数据清空，加载下一批
                imgs_list = []
                labels_list = []
        
        # 对于小于BATCESIZE的
        if len(imgs_list)>0:
            yield np.array(imgs_list), np.array(labels_list)
    return data_generator 

In [20]:
# 网络的定义
class SoftmaxNet(object):
    def __init__(self, num_features, num_outputs):
        self.W = np.zeros((num_features, num_outputs))
        self.b = np.zeros((1, num_outputs))

        

    def softmax(self,z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / exp_z.sum(axis=1, keepdims=True)
    
    def cross_entropy_loss(self, y_true, y_pred):
        return -np.mean(np.log(y_pred[np.arange(len(y_true)), y_true.flatten().astype(int)]))

        
    
    def train(self,epoch, lr=0.1):
        losses = []
        train_loader = load_data('train',batch_size=20)
        for epoch_id in range(epoch):
            for batch_id, data in enumerate(train_loader()):
                images, labels = data
                labels = labels.reshape(20,1)
                z = np.dot(images, self.W) + self.b
                y_pred = self.softmax(z)
                loss = self.cross_entropy_loss(labels, y_pred)

                grade = y_pred
                grade[range(len(labels)), int(labels)] -= 1
                grade /= len(labels)

                dW = np.dot(images.T, grade)
                db = np.sum(grade, axis=0, keepdims=True)

                # 更新梯度：
                self.W -= lr * dW
                self.b -= lr * db

            if batch_id % 200 ==0:
                print(f'epoch{epoch_id}, batch{batch_id}, loss{loss}')

                losses.append(loss)
        return losses
    # 测试函数
    def test(self):
        test_loader = load_data('eval', batch_size=20)
        accuracy = 0
        total_samples = 0

        for data in test_loader():
            imgs, labels = data
            z = np.dot(imgs, self.W) + self.b
            y_test_pred = self.softmax(z)
            accuracy += np.sum(np.argmax(y_test_pred, axis=1) == labels)
            total_samples += labels.shape[0]

        accuracy = accuracy / total_samples
        print(f'accuracy: {accuracy * 100:.2f}%')


        

In [21]:
net = SoftmaxNet(784, 10)
net.train(20,0.1)
net.test()

加载数据从./mnist .json.gz
加载数据集完毕
加载的数据集数量50000


TypeError: only size-1 arrays can be converted to Python scalars